<a href="https://colab.research.google.com/github/HASSANSHAHZADGITHUBWEB/ARCH-TECH-INTERNSHIP-/blob/main/Build_A_Speech_to_Reasoning_Pipeline_With_Whisper_%26_Quantized_LLM_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1 — install dependencies
# Run this first. It installs whisper, HF transformers, accelerate, bitsandbytes, and a tokenizer lib.
!pip install -q --upgrade pip
!pip install -q openai-whisper
!pip install -q transformers accelerate bitsandbytes sentencepiece safetensors
# (Optional: faster tokenizers)
!pip install -q "huggingface-hub"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


imports + basic helpers

In [2]:
# Cell 2 — imports + basic helpers
import os, time, gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import whisper
from IPython.display import Audio, display


pload or point to an audio file

In [4]:
# Cell 3 — upload or point to an audio file
# Option A: use Colab upload widget
from google.colab import files
print("Upload an audio file (wav/mp3/m4a).")
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print("Uploaded:", audio_path)

# Optional: play
display(Audio(audio_path))


Upload an audio file (wav/mp3/m4a).


Saving WhatsApp Audio 2025-09-27 at 00.33.14_19272b94.mp3 to WhatsApp Audio 2025-09-27 at 00.33.14_19272b94.mp3
Uploaded: WhatsApp Audio 2025-09-27 at 00.33.14_19272b94.mp3


transcribe with Whisper (CPU or GPU)

In [5]:
# Cell 4 — transcribe with Whisper (CPU or GPU)
# The small/medium models are faster. Choose based on your GPU.
whisper_model = whisper.load_model("small")   # "tiny","base","small","medium","large"
print("Whisper model loaded:", whisper_model.device)

print("Transcribing...")
result = whisper_model.transcribe(audio_path, verbose=False)
transcription = result.get("text", "").strip()
print("Transcription:\n", transcription)



100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 46.1MiB/s]


Whisper model loaded: cuda:0
Transcribing...
Detected language: English


100%|██████████| 427/427 [00:00<00:00, 594.11frames/s]

Transcription:
 Hello, check 1,2,3,check 1,2,3


l 5 — model selection (Unsloth 4-bit) & memory considerations
Recommended: pick an Unsloth pre-quantized model from Hugging Face.
Example here — choose a model that exists and is 4-bit:
 "unsloth/Llama-3.1-8B-Instruct-unsloth-bnb-4bit"  (replace with a model you prefer)
 Note: Unsloth dynamic 4-bit quants are designed to be more accurate than standard bnb-4bit while keeping VRAM low.
 See Unsloth docs and their HF collection for available models. :contentReference[oaicite:1]{index=1}

In [6]:
# Cell 5 — model selection (Unsloth 4-bit) & memory considerations
# Recommended: pick an Unsloth pre-quantized model from Hugging Face.
# Example here — choose a model that exists and is 4-bit:
#   "unsloth/Llama-3.1-8B-Instruct-unsloth-bnb-4bit"  (replace with a model you prefer)
#
# Note: Unsloth dynamic 4-bit quants are designed to be more accurate than standard bnb-4bit while keeping VRAM low.
# See Unsloth docs and their HF collection for available models. :contentReference[oaicite:1]{index=1}

MODEL_ID = "unsloth/Llama-3.1-8B-Instruct-unsloth-bnb-4bit"  # <-- change if needed

print("Selected model:", MODEL_ID)


Selected model: unsloth/Llama-3.1-8B-Instruct-unsloth-bnb-4bit


— load tokenizer + quantized model (bitsandbytes 4-bit)
# This tries to load-in-4bit using transformers + bitsandbytes.

In [7]:
# Cell 6 — load tokenizer + quantized model (bitsandbytes 4-bit)
# This tries to load-in-4bit using transformers + bitsandbytes.
# If you run into CUDA OOM, switch to a smaller model or a different dtype.

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# load model in 4-bit
bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_compute_dtype": torch.float16,   # compute in fp16
    "device_map": "auto",
}

print("Loading quantized model (this can take a few minutes)...")
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **bnb_config)
model.eval()
torch.cuda.empty_cache()
print("Model loaded. Allocated devices:", model.hf_device_map if hasattr(model, "hf_device_map") else None)


Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Model loaded. Allocated devices: {'': 0}


# — helper: generate answer from transcription

In [8]:
# Cell 7 — helper: generate answer from transcription
def ask_model(prompt, max_new_tokens=256, temperature=0.2, top_p=0.95):
    gen_cfg = GenerationConfig(
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=False
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, generation_config=gen_cfg)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # Sometimes model returns prompt+answer: strip prompt
    if prompt.strip() and text.startswith(prompt.strip()):
        answer = text[len(prompt):].strip()
    else:
        answer = text.strip()
    return answer


craft a reasoning prompt and run E2E

In [9]:
# Cell 8 — craft a reasoning prompt and run E2E
prompt_template = """
You are a careful reasoning assistant. Given the transcribed user audio below, answer the user's question or perform the requested reasoning step-by-step.

Transcript:
\"\"\"{trans}\"\"

Provide a short answer followed by concise chain-of-thought style steps (2-5 bullets).
"""

prompt = prompt_template.format(trans=transcription)
print("Prompt preview:\n", prompt[:600], "...\n")

start = time.time()
answer = ask_model(prompt, max_new_tokens=256, temperature=0.0)
end = time.time()
print(f"\nModel answer (runtime {end-start:.1f}s):\n")
print(answer)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Prompt preview:
 
You are a careful reasoning assistant. Given the transcribed user audio below, answer the user's question or perform the requested reasoning step-by-step.

Transcript:
"""Hello, check 1,2,3,check 1,2,3""

Provide a short answer followed by concise chain-of-thought style steps (2-5 bullets).
 ...


Model answer (runtime 59.8s):

You are a careful reasoning assistant. Given the transcribed user audio below, answer the user's question or perform the requested reasoning step-by-step.

Transcript:
"""Hello, check 1,2,3,check 1,2,3""

Provide a short answer followed by concise chain-of-thought style steps (2-5 bullets).
"""

## Step 1: Identify the pattern in the given sequence
The sequence provided is "check 1,2,3, check 1,2,3". This suggests a repetitive pattern where the numbers 1, 2, and 3 are checked in a sequence.

## Step 2: Determine the nature of the sequence
The sequence appears to be a simple counting sequence where the numbers 1, 2, and 3 are checked in a repeat

 cleanup tips (free VRAM between runs)

In [10]:
# Cell 9 — cleanup tips (free VRAM between runs)
del model
gc.collect()
torch.cuda.empty_cache()
print("Cleaned up model from GPU. Re-load when you need again.")


Cleaned up model from GPU. Re-load when you need again.


In [12]:
import nbformat

# Replace this with the actual path to your notebook file
path = "/content/your_notebook.ipynb"  # adjust path - REPLACE THIS COMMENT WITH YOUR NOTEBOOK PATH
nb = nbformat.read(path, as_version=4)

# Remove widgets metadata
if "widgets" in nb["metadata"]:
    nb["metadata"].pop("widgets")

nbformat.write(nb, path)
print("Notebook cleaned and ready to push to GitHub.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/your_notebook.ipynb'

In [13]:
import os
print(os.listdir('/content'))

['.config', 'WhatsApp Audio 2025-09-27 at 00.33.14_19272b94.mp3', 'WhatsApp Audio 2025-09-27 at 00.33.14_19272b94.dat.unknown', 'sample_data']
